# THIS PROJECT HAD DONE BY :
# MUSTAFA EMİR PEKER               NO:2200356011
# YUSUF KOCA                                NO:2200356013

# BBM409 FALL 2022 ASSIGNMENT 2

For this project we run our code in main() function. Also the pruning function is in the same class with the other functions because of that we can not split pruning function into a different cell in notebook to give seperate explanation.

We explained our code before our code blocks. All of our report's content given as explaination above our code blocks. 

Libraries that we imported:

In [176]:
import math

import pandas as pd
import numpy as np

Global variables:

In [177]:
global total_entropy
global data_frame

is_discrete: This function determines that wheter a column is discrete or not. It determines the discreteness of a column by looking its unique values. If the unique values have more than 10 members and all of its members are not  string than it assumes that the column is discrete.

In [178]:
#is it discrete or continuous
def is_discrete(data, col):
    unique = np.unique(data[:, col])
    # if there are more than 10 unique values and these values are not strings then it is continuous
    if len(unique) > 10 and not isinstance(unique[0], str):
        return False
    else:
        return True


make_it_discrete_from_cont: This function takes a column and makes the column's values discrete by getting the max and min of the column. After that it calculates range and step of the column. Then it compares column's according row value with the intervals. After comparing it converts the value according to intervals. We chose to divide the range by 10. First we tried dividing by 2 , it performed better than when we chose to use all continous as attribute values. With some try and error we decided that dividing by 10 performs the best for the first part. For the pruning part choosing even number of attributes might be a problem. Because when we are taking majority of the attribute values there can be a tie between values. 


In [179]:
def make_it_discrete_from_cont(data, col):

    # make countinous data discrete by taking the min and max of the column divide it into 10 equal parts
    min = np.min(data[:, col])
    max = np.max(data[:, col])
    # get the range
    range1 = max - min
    # get the step
    step = range1 / 10
    # loop through the column
    for i in range(len(data[:, col])):
        # if the value is less than the mean replace it with 0 else replace it with 1
        if data[i, col] < min + step:
            data[i, col] = 0
        elif data[i, col] < min + 2 * step:
            data[i, col] = 1
        elif data[i, col] < min + 3 * step:
            data[i, col] = 2
        elif data[i, col] < min + 4 * step:
            data[i, col] = 3
        elif data[i, col] < min + 5 * step:
            data[i, col] = 4
        elif data[i, col] < min + 6 * step:
            data[i, col] = 5
        elif data[i, col] < min + 7 * step:
            data[i, col] = 6
        elif data[i, col] < min + 8 * step:
            data[i, col] = 7
        elif data[i, col] < min + 9 * step:
            data[i, col] = 8
        else:
            data[i, col] = 9

    return data


make_all_discrete: This function combines make_it_discrete_from_cont and is_discrete. It takes whole data set and makes all columns discrete that are suitable for discretization process.

In [180]:
def make_all_discrete(data):
    # loop through the columns
    for i in range(len(data[0]) - 1):
        # check if the column is continous
        if (is_discrete(data, i) == False):
            # make the column discrete
            if (isinstance(data[0][i], str) == False):
                data = make_it_discrete_from_cont(data, i)
    return data

 remove_attrition:For simplicity reasons we changed the position of the 'Attrition' column to the end. 
 

In [181]:
def remove_attrition(data):
    # get the column indexed as 1
    attrition = data[:, 1]
    # remove the column indexed as 1
    data = np.delete(data, 1, 1)
    # append the column indexed as 1 to the end of the data set
    data = np.append(data, attrition[:, None], axis=1)
    return data

shuffle: A function to shuffle the data

In [182]:
# shuffle function from scratch
def shuffle(aux, n):
    # shuffle
    for i in range(n):
        # generate random index
        index = np.random.randint(0, len(aux))
        # swap
        aux[i], aux[index] = aux[index], aux[i]

    return aux

split_data: This function splits the data according to 5cv. After splitting it returns 5 training and 5 test data.

In [183]:
# split data into 5 cross validation train and test sets
def split_data(data, k):
    shuffle(data, len(data))
    split = np.array_split(data, k)
    # concatenate the split data
    training_set1 = np.concatenate((split[0], split[1], split[2], split[3]), axis=0)
    test_set1 = split[4]

    training_set2 = np.concatenate((split[0], split[1], split[2], split[4]), axis=0)
    test_set2 = split[3]

    training_set3 = np.concatenate((split[0], split[1], split[3], split[4]), axis=0)
    test_set3 = split[2]

    training_set4 = np.concatenate((split[0], split[2], split[3], split[4]), axis=0)
    test_set4 = split[1]

    training_set5 = np.concatenate((split[1], split[2], split[3], split[4]), axis=0)
    test_set5 = split[0]

    return [training_set1, test_set1], [training_set2, test_set2], [training_set3, test_set3], [training_set4,
                                                                                                test_set4], [training_set5, test_set5]


total_entropy: This function calculates total entropy of a given set. This function is used to calculate information gain .

In [184]:
# calculate entropy of a given data set numpy array with the last index being the attrition column with Yes or No
def total_entropy(data):
    # get the number of rows in the data set
    n = len(data)

    # get the number of rows where attrition is yes
    yes = len(data[data[:, -1] == 'Yes'])
    # get the number of rows where attrition is no
    no = len(data[data[:, -1] == 'No'])
    # calculate the entropy
    if yes == 0 or no == 0:
        return 0
    else:
        return -((yes / n) * np.log2(yes / n) + (no / n) * np.log2(no / n))


unique_values: This function takes a column and finds the unique values in the column. After that it calculates the entropy of the given column's unique values(i.e. it calculates the entropy of an attribute's unique values).After that it returns a dictionary where unique values are the keys and their entropy values are the... values:))

In [185]:
def unique_values(data, col):
    # get the unique values in the column
    unique = np.unique(data[:, col])
    # create a dictionary to store the unique values and their corresponding entropy
    unique_dict = {}
    # loop through the unique values
    for i in unique:
        # get the number of rows where the unique value is present
        n = len(data[data[:, col] == i])
        # get the number of rows where the unique value is present and attrition is yes
        yes = len(data[(data[:, col] == i) & (data[:, -1] == 'Yes')])
        # get the number of rows where the unique value is present and attrition is no
        no = len(data[(data[:, col] == i) & (data[:, -1] == 'No')])
        # calculate the entropy
        if yes == 0 or no == 0:
            unique_dict[i] = 0
        else:
            unique_dict[i] = -((yes / n) * np.log2(yes / n) + (no / n) * np.log2(no / n))
    return unique_dict


attribute_vars : This function returns the unique values of a given column

In [186]:
def attribute_vars(data, col):
    unique = np.unique(data[:, col])
    # get keys in an array
    keys = []
    for i in unique:
        keys.append(i)
    return keys


information_gain: This function calculates information gain of given column using previously defined functions.

In [187]:
def information_gain(data, col):
    # get the total entropy of the data set
    total = total_entropy(data)
    # get the unique values and their corresponding entropy
    unique_dict = unique_values(data, col)
    # get the number of rows in the data set
    n = len(data)
    # calculate the information gain
    for key, value in unique_dict.items():
        unique_dict[key] = (len(data[data[:, col] == key]) / n) * value
    return total - sum(unique_dict.values())


info_gain_max: This function takes a numpy array. It runs through all the columns of the numpy array. It calculates each column's information gain using information_gain function. After that it returns most informative attribute.

In [188]:
# calculate all info gains and return the column index with the highest info gain
def info_gain_max(data):
    # create a dictionary to store the column index and its corresponding info gain
    info_gain_dict = {}
    # loop through the columns
    for i in range(len(data[0]) - 1):
        # calculate the info gain of the column
        info_gain_dict[i] = information_gain(data, i)
    # return the column index with the highest info gain

    return max(info_gain_dict, key=info_gain_dict.get)


getNameByIndex: This function takes an index and returns initial attribute's name.
getIndexByName: This function takes a name and returns the index of a given name according to initial data.

In [189]:
def getNameByIndex(index):
    list = ['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Attrition']
    return list[index]

def getIndexByName(name):
    list = ['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Attrition']
    return list.index(name)


We have two types of nodes. For that we created two classes. 
AttributeNode class : This is the main node for the our tree it holds attributes and its childs.
LeafNode class: This is the leaf node that holds the value for the desicion. It holds an attribute that indicates its parent node.

In [190]:
# make a node class
class AttributeNode:
    def __init__(self, attribute,children):
        self.attribute = attribute
        self.children = children



class LeafNode:
    def __init__(self, value, attribute):
        self.value = value
        self.attribute = attribute



Here we have the DecisionTree class. This class has all the functions that needed for building a tree, making prediction out of a tree, pruning a tree and calculating the classification matrix.

build_tree: This function takes a dataset to build a tree.
            1-) It takes the most informative column and calculates its unique values.
            2-) Loops through its unique values and takes the matching rows with its current unique value.
            3-) After that it deletes the firstly calculated most informative column from dataset.
            4-) After deletion it checks if the dataset is pure(fully informative)
            5-) If it is pure it creates a leaf node that has the value of the fully informative dataset's 
                attrition value. And appends the leaf to the tree.
            6-) If it is not pure then function calls itself with the not-pure dataset and goes back to the first
                step. After all the recursive calls it creates a node and appends to the tree.
            7-) After looping throuh unique values it creates a node with the highest informative attribute and it 
                appends node to the tree.
            8-) After calculating all the nodes in the tree it returns the root node of the tree.

predict: This function takes a row and it guesses the attrition value of the row by roaming the according to row's 
         attribute values. It returns the guessed attrition value.

accuracy: This function takes a test set and compares its row's attrition values with the predicted values. After             that it calculates the metrices that are needed for evaluating the tree's success and returns them.

find_twigs: This function finds all the nodes whose children are all leafs.It does that roaming through allmost all             nodes and checking their children. It returns a list of twigs.

find_parent: This function checks all the nodes by recursively . If it finds the given node it understands that the              previous node that called the current recursion is the parent. After that it returns the parent node.

get_majority: This function takes a twig node and it calculates the majority value among its children. If the
              values are equal in numbers then it returns the first value it encountered with. Otherwise it returns
              the majority value.
             
entropy: This function calculates the entropy of a given twig.
              
information_gain: This function calculates the information gain of a given twig using the entropy function.

prune: This function prunes the tree by following these steps:
        Step 1: Catalog all twigs in the tree
        Step 2: Find the twig with the least Information Gain
        Step 3: Remove all child nodes of the twig
        Step 4: Relabel twig as a leaf (Set the majority of ”Positive” or ”Negative” as leaf value)
        Step 5: Measure the accuracy value of your decision tree model with removed twig on the validation set 
        ("Current Accuracy")
        If ”Current Accuracy ≥ Last Accuracy” : Jump to ”Step1”
        Else : Revert the last changes done in Step 3,4 and then terminate


In [191]:
class DecisionTree:

    def build_tree(self, data):
        # get the column index with the highest info gain
        max_info_gain = info_gain_max(data)
        # get the unique values in the column
        unique = attribute_vars(data, max_info_gain)
        # create a list to store the children nodes
        children = []
        # loop through the unique values
        for i in unique:
            # get the rows where the unique value is present
            rows = data[data[:, max_info_gain] == i]
            # remove the column with the highest info gain
            rows = np.delete(rows, max_info_gain, 1)
            # check if the rows are pure
            if (is_pure(rows)):
                # create a leaf node
                leaf = LeafNode(rows[0][-1], i)
                # append the leaf node to the children list

                children.append(leaf)
            else:
                # create a new attribute node
                node = AttributeNode(i, self.build_tree(rows))
                # append the node to the children list
                children.append(node)
        # create an attribute node
        node = AttributeNode(getNameByIndex(max_info_gain), children)
        return node


    # make a method that takes a row and returns the label
    def predict(self, row, node):
        # check if the node is a leaf node
        if (isinstance(node, LeafNode)):
            return node.value
        else:
            # get the attribute
            attribute = node.attribute
            # loop through the children

            if (isinstance(node.children, AttributeNode)):
                # take the node.children's children
                children = node.children.children
                node_attribute = node.children.attribute

                node1 = AttributeNode(node_attribute, children)
                node = node1
                attribute = node.attribute


            for i in node.children:
                # check if the attribute is equal to the child's attribute
                # print(row[getIndexByName(attribute)])
                if (row[getIndexByName(attribute)] == i.attribute):
                    # return the label
                    return self.predict(row, i)



    def accuracy(self, data, node):
        # create a list to store the predictions
        predictions = []
        tp = 0
        tn = 0
        fp = 0
        fn = 0

        # loop through the data
        for i in data:
            # get the prediction
            prediction = self.predict(i, node)
            # append the prediction to the list
            predictions.append(prediction)
        # get the number of correct predictions
        correct = 0
        # loop through the predictions
        for i in range(len(predictions)):
            # check if the prediction is correct
            if (predictions[i] == data[i][-1]):
                # increment the correct predictions
                correct += 1
            if ( predictions[i] == 'Yes' and data[i][-1] == 'Yes'):
                tp += 1
            if ( predictions[i] == 'No' and data[i][-1] == 'No'):
                tn += 1
            if ( predictions[i] == 'Yes' and data[i][-1] == 'No'):
                fp += 1
            if ( predictions[i] == 'No' and data[i][-1] == 'Yes'):
                fn += 1


        # calculate the accuracy
        accuracy = correct / len(predictions)

        # calculate the precision
        precision = tp / (tp + fp)

        # calculate the recall
        recall = tp / (tp + fn)

        # calculate the f1 score
        f1 = 2 * (precision * recall) / (precision + recall)

        # return all the values
        return accuracy, precision, recall, f1


    # find all the twigs in the tree. The twigs are the nodes whose children are all leaves.
    def find_twigs(self, node):
        # create a list to store the twigs
        twigs = []
        # check if the node is a leaf node
        if (isinstance(node, LeafNode)):
            return twigs
        # check if the node's children are all leaf nodes
        if (isinstance(node.children, AttributeNode)):
            # take the node.children's children
            children = node.children.children
            node_attribute = node.children.attribute

            node1 = AttributeNode(node_attribute, children)
            node = node1

        if (all(isinstance(i, LeafNode) for i in node.children)):
            # append the node to the twigs list
            twigs.append(node)
        else:
            # loop through the children
            for i in node.children:
                # get the twigs from the child
                twig = self.find_twigs(i)
                # append the twig to the twigs list
                twigs.extend(twig)
        return twigs


    # find parent of a twig
    def find_parent(self, node, twig):
        # check if the node is a leaf node

        if (isinstance(node, LeafNode)):
            return None
        # check if the node's children are all leaf nodes
        if (isinstance(node.children, AttributeNode)):
            # take the node.children's children
            children = node.children.children
            node_attribute = node.children.attribute

            node1 = AttributeNode(node_attribute, children)
            node = node1

        if (all(isinstance(i, LeafNode) for i in node.children)):
            # compare the node' attribute with the twig's attribute and compare the node's children with the twig's children
            if (node.attribute == twig.attribute):
                if (all(node.children[i].attribute == twig.children[i].attribute for i in range(len(node.children)))):
                    return node
            return None
        else:
            # loop through the children
            for i in node.children:
                # check if the child is the twig

                if (i == twig):
                    # return the node
                    return node
                # get the parent from the child
                parent = self.find_parent(i, twig)
                # check if the parent is not None
                if (parent is not None):

                    return parent


    def get_majority(self, node):
        # create a list to store the labels
        labels = []
        # loop through the children
        for i in node.children:
            # append the label to the list
            labels.append(i.value)
        # get the most common label

        # if there are more than one label with the same frequency, return the first one
        if (labels.count(max(set(labels), key=labels.count)) > 1):
            return labels[0]


    def entropy(self, positive, negative):
        # calculate the entropy
        total = positive + negative
        entropy = 0
        if (positive != 0):
            entropy += -(positive / total) * math.log2(positive / total)
        if (negative != 0):
            entropy += -(negative / total) * math.log2(negative / total)
        return entropy

    def information_gain(self, node):
        # get the number of positive and negative labels
        positive = 0
        negative = 0
        # loop through the children
        for i in node.children:
            # check if the label is positive
            if (i.value == "Yes"):
                # increment the positive count
                positive += 1
            else:
                # increment the negative count
                negative += 1
        # calculate the information gain
        info_gain = 1 - self.entropy(positive, negative)
        return info_gain


    def prune(self, node, validation_data, last_accuracy):
        # find all the twigs in the tree
        twigs = self.find_twigs(node)
        # check if the twigs list is empty
        if (len(twigs) == 0):
            return node
        # find the twig with the least information gain
        twig = min(twigs, key=lambda x: self.information_gain(x))
        # find the parent of the twig
        parent = self.find_parent(node, twig)
        # remove all the children of the twig
        majority = self.get_majority(twig)
        twig.children = []
        # relabel the twig as a leaf
        twig.children.append(LeafNode(majority, twig.attribute))
        # measure the accuracy of the tree
        current_accuracy = self.accuracy(validation_data, node)[0]
        # check if the current accuracy is greater than or equal to the last accuracy
        if (current_accuracy > last_accuracy):
            # prune the tree
            print("Pruning : " + str(current_accuracy))
            return self.prune(node, validation_data, current_accuracy)
        else:
            # revert the changes
            twig.children = parent.children
            return node





is_pure: This method checks if a given set is full informative or not.

In [192]:
# is pure method
def is_pure(data):
    # get the number of rows in the data set
    n = len(data)
    # get the number of rows where attrition is yes
    yes = len(data[data[:, -1] == 'Yes'])
    # get the number of rows where attrition is no
    no = len(data[data[:, -1] == 'No'])
    # check if the data set is pure
    if yes == n or no == n:
        return True
    else:
        return False


And this is the main function that we run our code on.

In [193]:

# create main method
def main():
    # read csv file and convert it into numpy array
    df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')

    names_of_columns = df.columns

    names_of_columns = names_of_columns.tolist()

    # remove the attrition column from the list of column names
    names_of_columns.remove('Attrition')
    # append the attrition column to the end of the list of column names
    names_of_columns.append('Attrition')

    data = df.values

    data = remove_attrition(data)

    data = make_all_discrete(data)

    data = shuffle(data, len(data))

    splitted_data = split_data(data,5)

    avg_accuracy = 0.0
    avg_precision = 0.0
    avg_recall = 0.0
    avg_f1 = 0.0


    dt = DecisionTree()

    for i in range(5):
        train_data = splitted_data[i][0]
        test_data = splitted_data[i][1]

        node = dt.build_tree(train_data)
        avg_accuracy += dt.accuracy(test_data, node)[0]
        avg_precision += dt.accuracy(test_data, node)[1]
        avg_recall += dt.accuracy(test_data, node)[2]
        avg_f1 += dt.accuracy(test_data, node)[3]


    avg_accuracy /= 5
    avg_precision /= 5
    avg_recall /= 5
    avg_f1 /= 5


    print("Average Accuracy: ", avg_accuracy)
    print("Average Precision: ", avg_precision)
    print("Average Recall: ", avg_recall)
    print("Average F1: ", avg_f1)
    
    pruning_validation = data[0:294]
    










    dt.prune(node, pruning_validation, avg_accuracy)










In [194]:
if __name__ == '__main__':
    main()


Average Accuracy:  0.7149659863945578
Average Precision:  0.7429158918229637
Average Recall:  0.765911639188235
Average F1:  0.7481856581135096
Pruning : 0.7721088435374149


Here we average accuracy, precision, recall, f1 scores and accuracy score after pruning . We ran this code several times and we can say that our model has average accuracy between 70-90 percent. Usually after pruning our accuracy level increases between 10 percent.

